# Generate the transitive closure table for the NCI Thesaurus

This notebook shows how to generate the transitive closure table for the NCI (National Cancer Institute) Thesaurus. 
It uses a combination of sqlite and pandas to minimize the amount of code needed to create the transitive closure table.  

Download the latest flat format thesaurus file from here: https://cbiit.cancer.gov/evs-download/thesaurus-downloads
Unzip the file into a directory, and set the full path to the name of the Thesaurus.txt fle.

The NCI Thesaurus (NCIt) is a multi-axial hierarchy.  If a node has more than one parent, then the parents column will contain the C codes for the parents in a pipe delimited string.

Hubert Hickman

In [1]:
import sqlite3
import pandas
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [35]:
# Set these to whatever you need them to be.
thesaurus_file = '/Volumes/workdisk/ncit/Thesaurus.txt'
database_file = '/Volumes/workdisk/ncit/nci.db'

Names in the dataframe will become columns in the sqlite database

In [37]:
ncit_df = pandas.read_csv(thesaurus_file, delimiter = '\t', header = None, 
                         names=('code', 'url', 'parents','synonyms', 
                                'definition', 'display_name', 'concept_status', 'semantic_type', 'pref_name' ))

Add in the preferred name field - the first choice in the list of synonyms.  Note that this column is not in the NCI Thesaurus and can be computed in pandas or in sqlite.

In [38]:
ncit_df['pref_name'] = ncit_df.apply(lambda row: row['synonyms'].split('|')[0], axis=1)

In [39]:
print(ncit_df)

           code                                                url  parents  \
0       C100000  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C49803   
1       C100001  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C49803   
2       C100002  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C49803   
3       C100003  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C80449   
4       C100004  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C80430   
5       C100005  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C15839   
6       C100006  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...  C139982   
7       C100007  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C99896   
8       C100008  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C49803   
9       C100009  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C99896   
10       C10000  <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...   C61007   
11      C100010  <http://ncicb.nci.nih.gov/xml/owl/E

In [40]:
con = sqlite3.connect(database_file)

In [41]:
# Write the dataframe out to the sqlite table
ncit_df.to_sql('ncit', con=con, if_exists = 'replace')

In [42]:
con.execute("drop index if exists ncit_code_index")
con.execute("create index ncit_code_index on ncit(code)")
con.commit()

In [43]:
cur=con.cursor()
cur.execute("select code, parents from ncit where parents is not null")

Get all of the concepts that have parents into a result set 

In [44]:
concept_parents = cur.fetchall()

In [45]:
print(len(concept_parents))

145790


In [46]:
con.commit()

Create a table that will hold the concept, the parent, the path from parent to the concept, and the level (need this to properly recurse along the relationship).

In [47]:
con.execute('drop table if exists parents')
con.execute("""
create table parents (
concept text,
parent text,
path text,
level int)
""")

Put the direct concept &rarr; parent relationships in the table as level 1 items.

In [48]:
for concept, parents in concept_parents:
    parentl = parents.split('|')
    for p in parentl:
        con.execute("insert into parents(concept, parent, level, path )values(?,?,1,?)", (concept,p,p +'|'+ concept))
con.commit()

    

In [49]:
print(con.execute("select * from parents where parent= 'C3824'").fetchall())

[('C100063', 'C3824', 'C3824|C100063', 1), ('C110961', 'C3824', 'C3824|C110961', 1), ('C110962', 'C3824', 'C3824|C110962', 1), ('C111029', 'C3824', 'C3824|C111029', 1), ('C111971', 'C3824', 'C3824|C111971', 1), ('C111972', 'C3824', 'C3824|C111972', 1), ('C117372', 'C3824', 'C3824|C117372', 1), ('C120880', 'C3824', 'C3824|C120880', 1), ('C121146', 'C3824', 'C3824|C121146', 1), ('C123253', 'C3824', 'C3824|C123253', 1), ('C124069', 'C3824', 'C3824|C124069', 1), ('C126089', 'C3824', 'C3824|C126089', 1), ('C126781', 'C3824', 'C3824|C126781', 1), ('C128698', 'C3824', 'C3824|C128698', 1), ('C139013', 'C3824', 'C3824|C139013', 1), ('C139024', 'C3824', 'C3824|C139024', 1), ('C142817', 'C3824', 'C3824|C142817', 1), ('C155726', 'C3824', 'C3824|C155726', 1), ('C35935', 'C3824', 'C3824|C35935', 1), ('C35936', 'C3824', 'C3824|C35936', 1), ('C36109', 'C3824', 'C3824|C36109', 1), ('C36110', 'C3824', 'C3824|C36110', 1), ('C36164', 'C3824', 'C3824|C36164', 1), ('C36298', 'C3824', 'C3824|C36298', 1), ('C

In [50]:
con.execute("drop index if exists par_concept_idx")
con.execute("create index par_concept_idx on parents(concept)")
con.execute("drop index if exists par_par_idx")
con.execute("create index par_par_idx on parents(parent)")

This is the key part - execute the recursive SQL to generate the set of all paths through the NCIt.  We'll prune this to just concepts and descendants a few steps below.  

In [63]:
con.execute("drop table if exists ncit_tc_with_path")
con.execute('create table ncit_tc_with_path as with recursive ncit_tc_rows(parent, descendant, level, path ) as ' + 
'(select parent, concept as descendant, level, path from parents union all '+ 
"select p.parent , n.descendant as descendant, n.level+1 as level ,  p.parent || '|' || n.path  as path " +
'from ncit_tc_rows n join parents p on n.parent = p.concept  ' + 
') select * from ncit_tc_rows')

con.execute('create index ncit_tc_path_parent on ncit_tc_with_path(parent)')
con.commit()

Create the transitive closure table.  This fits the mathematical definition of transitive closure. 

In [52]:
con.execute('drop table if exists ncit_tc')
con.execute("create table ncit_tc as select distinct parent, descendant from ncit_tc_with_path ")
con.execute('create index ncit_tc_parent on ncit_tc (parent) ')

In [53]:
print(con.execute('select count(*) from ncit_tc').fetchone())

(989181,)


In [54]:
pp.pprint(con.execute("select count(*) from ncit_tc where parent = 'C7057'").fetchall())

[(36271,)]


In [55]:
pp.pprint(con.execute("select code from ncit where parents is null").fetchall())

[   ('C12218',),
    ('C12219',),
    ('C12913',),
    ('C14250',),
    ('C16612',),
    ('C17828',),
    ('C1908',),
    ('C20047',),
    ('C20181',),
    ('C20189',),
    ('C20633',),
    ('C22187',),
    ('C22188',),
    ('C26548',),
    ('C28389',),
    ('C28428',),
    ('C3910',),
    ('C43431',),
    ('C7057',),
    ('C97325',)]


In [56]:
pp.pprint(con.execute("select * from ncit_tc limit 10").fetchall())

[   ('C49803', 'C100000'),
    ('C49803', 'C100001'),
    ('C49803', 'C100002'),
    ('C80449', 'C100003'),
    ('C80430', 'C100004'),
    ('C15839', 'C100005'),
    ('C139982', 'C100006'),
    ('C99896', 'C100007'),
    ('C49803', 'C100008'),
    ('C99896', 'C100009')]


We need to add in the reflexive relations (i.e. xRx for any concept x).  This is a convenience to simplify the subsumption operations using the database table.

In [57]:
con.execute(
'''with codes as 
(
select distinct parent as code from ncit_tc
union
select distinct descendant as code from ncit_tc
) 
insert into ncit_tc (parent, descendant) 
select c.code as parent, c.code as descendant from codes c
''')

In [64]:
con.execute(
'''with codes as 
(
select distinct parent as code from ncit_tc
union
select distinct descendant as code from ncit_tc
) 
insert into ncit_tc_with_path (parent, descendant, level, path) 
select c.code, c.code, 0  , c.code  from codes c
''')

In [59]:
reflexive_concepts  = con.execute("select count(*) from ncit_tc where parent=descendant").fetchall()

In [60]:
print(reflexive_concepts)

[(145809,)]


In [61]:
con.execute("select count(*) from ncit_tc ").fetchall()

[(1134990,)]

In [62]:
con.execute("drop index if exists tc_parent_index")
con.execute("create index tc_parent_index on ncit_tc(parent)")

At this point, several options are available. 
* Use this table in operations.
* Export the table in CSV to use in other situations
* Reinstantiate the table as a pandas dataframe to use directly in Python/pandas

In [65]:
con.commit()